# Cargamos algunas librerias que vamos a utilizar luego

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from src.utils import load_dataset, split_data, DataLoader
from src.text import standardize_method

# Indice orden:

- (1) Presentación de oportunidad. (Cubiertos en la sesion de metodologia)
- (2) Medición del posible impacto. (Cubiertos en la sesion de metodologia)
- (3) Criterios de éxito:
    - 3.a Métricas del modelo
    - 3.b Métricas de negocio (Cubiertos en la sesion de metodologia)
- (4) Recolección de datos y prueba de hipótesis (Partimos de datos)
- (5) Exploración de los datos. 
- (6) Modelo de base.
- (7) Prototipado.
- (8) Encontrado la arquitectura -> búsqueda de hiper-parámetros -> Entrenamiento. (Por ahora fuera del alcance)
- (9) Despliegue en producción. (Por ahora fuera del alcance)
- (10) Monitoreo. (Por ahora fuera del alcance)
- (11) A/B testing. (Por ahora fuera del alcance)
- (12) Concluciones con respecto a los criterios de exito -> reunion con stakeholders. (Por ahora fuera del alcance)


# Cargamos los datos

In [ ]:
dataset = load_dataset("../datasets/awzm_products.jsonl.gz")

In [ ]:
dataset.info()

# (3) Criterios de éxito

# Balance

Sabemos que el problema se puede clasificar como un problema de multiclass donde un producto puede pertenecer a un numero de clases

Es nuestro dataset balanceado?

In [ ]:
main_categories = dataset.main_cat.unique()

In [ ]:
main_cat_count_df = dataset.groupby(by="main_cat").main_cat.count().reset_index(name="count").sort_values(by="count", ascending=True)

main_cat_count_df

In [ ]:
main_categories_count = dataset.main_cat.unique().shape[0]

equal_representation_count = int(dataset.shape[0]/main_categories_count)

print(f"Tenemos : {main_categories_count} Si estruviera balanceado esperariamos ver {equal_representation_count} samples por categoria")

## Porcentaje de representacion con respecto a lo esperado

In [ ]:
main_cat_count_df["%"] = main_cat_count_df["count"]/equal_representation_count
main_cat_count_df["diferencia%"] = main_cat_count_df[["%"]]-1

main_cat_count_df.sort_values(by="diferencia%", ascending=True)

## 3. Metricas de exito sobre el rendimiento del modelo:

Sabemos que no se encuentra balanceado con lo cual tenemos que tomar una metrica de evaluación que tenga encuenta ese desbalance en este caso vamos utilizar ***F1 Score weighted*** para evaluar los modelos.

## 3b. Como metrica de negocio:
Nos interesa entender el costo de las desiciones que toma el modelo. Y cuanto tiempo estamos reduciendo.

```Omitimos los pasos 4 y 5 ya que el dataset lo tenemos listo para entrenar.```

## (5) Exploración de los datos.

En principio no vamos a tener mucho tiempo para esta parte pero sepan que es donde vemos que features podemos usar y cual seria la mejor forma de procesarlas para que el modelo las pueda usar.

Vamos a partir de que el titulo es nuestro primer candidato por varias razones:
- Es un campo obligatorio
- Cumple el fin de resumir y expresar al mismo tiempo la mayor cantidad de informacion posible sobre el producto

# (6) Modelo base (baseline)

### Split de los datos:

Primero vamos a separa los datos que tenemos utilizando una division de 98% training /1% validacion y /1% testing

In [ ]:
(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = split_data(dataset=dataset)

## Instrucciones

Aqui pueden idear el baseline que deseen, puede ser algo completamente random, pueden predecir siempre una clase quizas la mas representada por ejemplo o algun otro que se les ocurra.
El punto importante es tener algo sobre lo cual comparar el resultado del primer modelo que va a ser el mas sensillo pero que aun asi deberia superar ampliamente al baseline

Esta prueba de como funciona un baseline lo vamos a hacer sobre Y_val para poder comprar luego con nuestro modelo sobre el mismo set de datos.

In [ ]:
# recueden de no pisar el contenido de Y_val
import copy

# aqui van las predicciones de ustedes que pueden ser puro random o alguna clase en particular.
test_labels_copy = #


# Calculamos los aciertos
match_labels = Y_val == test_labels_copy

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_val, test_labels_copy, output_dict=False))

El numero que nos interesa es el que se encuentra en "weighted avg" y "f1-score"

# (7) prototipado

## Para realizar el prototipo tenemos que recordar algunas cosas

Es un problema de clasificación múltiple con lo cual es no nos dice que tipo de funcion de costo debemos usar y en segundo lugar la métrica a superar es la del modelo base en principio.
En caso de que no podamos superar esa metrica, debemos volver a las hipotesis y replantear el prototipo, quizas necesitemos otra arquitectura o un modelo mas grande.


En principio vamos a utilizar lo mas simple posible que es un modelo con un proceso de **multi_one_hot**. Pueden probar otro modelo u otro preproceso como tf-idf por ejemplo.

## pre Preproceso

Vamos a elimiar del titulo los emojis si es que hay y signos de puntuación ademas de reemplazar los numeros por un token que indica que hay un numero en ese lugar.

Para ello vamos a utilizar el **DataLoader** que nos va a proporcionar los sets de entrenamiento, validacion y testing.

En principio vamos a utilizar un vocabulario de 10000 tokens con un bache de 1024. Pueden modificarlo ampliar/reducir tanto el vocabulario como el batch_size, este ultimo va a determinar cuantos
samples se utilzan para entrenar en forma simultanea, si reciben un error de OOM pueden reducir. Si en cambio notan que su GPU no se encuentra con mucha carga pueden ampliarlo.

In [ ]:
data_loader = DataLoader(vocab_size=10000, classes=22, batch_size=1024)

data = data_loader.build_datasets("../datasets/awzm_products.jsonl.gz")

train_dataset = data["train"]
val_dataset = data["validation"]
test_dataset = data["test"]

 # Prototipo

 En un principio pueden elegir el modelo que quieran con las capas, unidades y activaciones que deseen

In [ ]:
model = tf.keras.Sequential([
    # Espacio para agregar las capas que crean necesarias
    # recuerden que si o si tienen que terminar con una salida de una funcion de densidad de probabilidad (softmax)
])

Para hacer el entrenamiento mas rapido y eficiente pueden usar los siguientes callbacks

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="temporal_checkpoint",
                                                      save_weights_only=True,
                                                      save_only_best=True,
                                                      verbose=1)

In [ ]:
model.compile(
  # Tienen que definir una loss function

  # puden usar el optimizer que deseen

  # la metrica va a ser F1-score weighted por lo que encontramos sobre el desbalance de las clases
  metrics=[tf.keras.metrics.F1Score(average="weighted", threshold=None, name='f1_score', dtype=None)]
)

## Hora de entrenar

Si todo salio bien ahora van a poder entrenar un modelo!

In [ ]:
model.fit(train_dataset,
            validation_data=val_dataset,
            epochs=500,
            callbacks=[model_checkpoint, early_stop]
        )